In [1]:
!pip install -U datasets huggingface_hub fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [2]:
import pandas as pd  # requires: pip install pandas
import torch
from chronos import BaseChronosPipeline, ChronosPipeline, ChronosTokenizer
import numpy as np
from datasets import load_dataset, Dataset
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq

/Users/rckyi/miniforge3/envs/gpu_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# return cls(
#             tokenizer=chronos_config.create_tokenizer(),
#             model=ChronosModel(config=chronos_config, model=inner_model),
#         )


pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",  # use "amazon/chronos-bolt-small" for the corresponding Chronos-Bolt model
    device_map="auto",  # use "cpu" for CPU inference
    torch_dtype=torch.bfloat16,
)

In [143]:
model = pipeline.model
tokenizer = pipeline.tokenizer

In [12]:
from huggingface_hub import notebook_login
notebook_login()

In [15]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
dataset = load_dataset("shaddie/thrust_curves_dataset")

Repo card metadata block was not found. Setting CardData to empty.


In [30]:
dataset

DatasetDict({
    train: Dataset({
        features: ['time', 'thrust', 'motorId', 'motor_name', 'impulse_class', 'id'],
        num_rows: 6008
    })
})

In [27]:
# dataset['train']['time']

In [ ]:
# dataset = load_dataset("monash_tsf", "weather")

In [ ]:
time_raw_values = [example['time'] for example in thrust_dataset['train']]
thrust_raw_values = [example['thrust'] for example in thrust_dataset['train']]
raw_values = [example['time'] for example in thrust_dataset['train']]

In [ ]:
def generate_windows(values, context_length, prediction_length):
    inputs, targets = [], []
    total_length = context_length + prediction_length
    for i in range(len(values) - total_length + 1):
        context = values[i : i + context_length]
        target = values[i + context_length : i + total_length]
        inputs.append(context)
        targets.append(target)
    return inputs, targets

In [ ]:
inputs, targets = generate_windows(raw_values, context_length, prediction_length)

In [47]:
from collections import defaultdict
from datasets import Dataset

context_length = 24
prediction_length = 12

# Step 1: Group values by 'motorId'
groups = defaultdict(list)
for example in dataset['train']:
    id = example['id']
    time = example['time']
    motor_id = example['motorId']
    value = example['thrust']
    impulse_class = example['impulse_class']
    groups[motor_id].append({'id': id, 'time': time, 'thrust': value, 'motor_id': motor_id, 'impulse_class': impulse_class})

In [48]:
groups['5f4294d200023100000000f7']

[{'id': 0,
  'time': 0.0,
  'thrust': 0.0,
  'motor_id': '5f4294d200023100000000f7',
  'impulse_class': 'J'},
 {'id': 1,
  'time': 0.0,
  'thrust': 1.0,
  'motor_id': '5f4294d200023100000000f7',
  'impulse_class': 'J'},
 {'id': 2,
  'time': 0.0533667938931297,
  'thrust': 0.800000899236099,
  'motor_id': '5f4294d200023100000000f7',
  'impulse_class': 'J'},
 {'id': 3,
  'time': 0.4013167938931297,
  'thrust': 0.6923083840277684,
  'motor_id': '5f4294d200023100000000f7',
  'impulse_class': 'J'},
 {'id': 4,
  'time': 0.7876946564885495,
  'thrust': 0.4153841311805621,
  'motor_id': '5f4294d200023100000000f7',
  'impulse_class': 'J'},
 {'id': 5,
  'time': 0.8239847328244275,
  'thrust': 0.2307687119791736,
  'motor_id': '5f4294d200023100000000f7',
  'impulse_class': 'J'},
 {'id': 6,
  'time': 0.8709465648854962,
  'thrust': 0.1538462576041652,
  'motor_id': '5f4294d200023100000000f7',
  'impulse_class': 'J'},
 {'id': 7,
  'time': 1.0,
  'thrust': 0.0,
  'motor_id': '5f4294d200023100000000f

In [92]:
group_keys = []
for key, value in groups.items():
    group_keys.append({'id': [v['id'] for v in value][0] 
                       ,'time':[(v['time']) for v in value]
                       ,'thrust':[(v['thrust']) for v in value]
                       ,'motor_id':key
                       ,'impulse_class': [(v['impulse_class']) for v in value][0]
                      })
    # print(f'key, value {key} -> {len(value)}')

In [105]:
# group_keys[1]

In [106]:
# group_keys[2]

In [49]:
# Step 2: Slide windows within each group
def generate_windows(values, context_length, prediction_length):
    total_length = context_length + prediction_length
    windows = []
    for i in range(len(values) - total_length + 1):
        context = values[i : i + context_length]
        target = values[i + context_length : i + total_length]
        windows.append((context, target))
    return windows

In [162]:
# Step 3: Aggregate all windows into a flat dataset
windowed_data = []
for motor_id, values in groups.items():
    windows = generate_windows(values, context_length, prediction_length)
    # print(f'motor id {windows}')
    for context, target in windows:
        windowed_data.append({
                    'input':{'id': [c['id'] for c in context][0] 
                            ,'time':[(c['time']) for c in context]
                            ,'thrust':[(c['thrust']) for c in context]
                            ,'motor_id':[(c['motor_id']) for c in context][0]
                            ,'impulse_class': [(c['impulse_class']) for c in context][0]
                            }
                    , 'target':{'id': [t['id'] for t in target][0] 
                            ,'time':[(t['time']) for t in target]
                            ,'thrust':[(t['thrust']) for t in target]
                            ,'motor_id':[(t['motor_id']) for t in target][0]
                            ,'impulse_class': [(t['impulse_class']) for t in target][0]
                            }
        })

In [163]:
# Step 4: Build a Dataset
windowed_dataset = Dataset.from_list(windowed_data)

In [164]:
windowed_dataset

Dataset({
    features: ['input', 'target'],
    num_rows: 191
})

In [165]:
windowed_dataset['input'][0]

{'id': 23,
 'impulse_class': 'J',
 'motor_id': '5f4294d20002310000000256',
 'thrust': [0.0,
  0.9454233664183034,
  0.7429588165385054,
  1.0,
  0.7183106043750306,
  0.711268064230788,
  0.5633801479326868,
  0.7429588165385054,
  0.8327467941586668,
  0.7376758939182626,
  0.7235921703125255,
  0.7411978423317578,
  0.7411978423317578,
  0.7447184340625052,
  0.7429588165385054,
  0.7605631318749899,
  0.7834517265144649,
  0.8028170160576971,
  0.8133801479326868,
  0.8116205304086872,
  0.8098595562019396,
  0.8045779902644447,
  0.7975354501202021,
  0.7799297781009698],
 'time': [0.0,
  0.0143426294820717,
  0.0191235059760956,
  0.0247011952191235,
  0.0310756972111553,
  0.0406374501992031,
  0.0430278884462151,
  0.050199203187251,
  0.0629482071713147,
  0.0749003984063745,
  0.0844621513944223,
  0.1011952191235059,
  0.1163346613545816,
  0.1521912350597609,
  0.2231075697211156,
  0.30199203187251,
  0.3768924302788844,
  0.4446215139442231,
  0.5011952191235061,
  0.54900

In [166]:
# windowed_dataset['target'][0]

In [180]:
def preprocess_windowed(example):
    # Include motorId in input prompt if helpful (optional)
    # input_str = f"forecast: " + " ".join(map(str, example["input"]))
    # target_str = " ".join(map(str, example["target"]))
    # print(example['input']['thrust'])
    t = torch.from_numpy(np.asarray(example['input']['thrust']))
    t_shape = t.shape
    t = t.reshape(1, t_shape[0])
    # print(t.shape)

    tokenized, attention_mask, scale = tokenizer.context_input_transform(t)
    
    # tokenizer.pad_token = tokenizer.eos_token
    return {
            "token_ids": tokenized
            , "attention_mask": attention_mask
            , "scale": scale
    }

tokenized_dataset = windowed_dataset.map(preprocess_windowed, remove_columns=windowed_dataset.column_names)

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 191/191 [00:00<00:00, 2777.01 examples/s]


In [181]:
tokenized_dataset

Dataset({
    features: ['token_ids', 'attention_mask', 'scale'],
    num_rows: 191
})

In [ ]:
# training_args = TrainingArguments(
#     output_dir="/thrust_curve_chronos_ts_model",
#     per_device_train_batch_size=8,
#     gradient_accumulation_steps=2,
#     num_train_epochs=5,
#     logging_steps=50,
#     save_steps=500,
#     save_total_limit=2,
#     learning_rate=2e-4,
#     bf16=torch.cuda.is_bf16_supported(),
#     fp16=not torch.cuda.is_bf16_supported(),
#     report_to="none"
# )

In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )

# # Step 5: Train
# trainer.train()
